<a href="https://colab.research.google.com/github/aditisinghq/book-talk/blob/main/contentbased_NPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
!pip install rake_nltk
#rake_nltk is the library i am  using to extract keywords from description!. it has stopwords (by default english ) preloaded

In [123]:
from rake_nltk import Rake
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
df = pd.read_csv('/content/drive/MyDrive/books7k.csv', engine='python', error_bad_lines=False)
df.head()
df=df[['isbn10','title','authors','thumbnail','description','categories']]
df.dropna(inplace = True)
print(df.shape)
df.drop_duplicates(subset=['isbn10'])#there aren't any duplicate isbns(there are some anamolies)
print(df.shape)
df['description'].apply(type).value_counts()
import string
df['description'] = df['description'].str.replace('[{}]'.format(string.punctuation), '')
df['description'].astype(str)
df['description'].apply(type).value_counts()



(6247, 6)
(6247, 6)


<class 'str'>    6247
Name: description, dtype: int64

In [124]:
df['Key_words'] = ''
r = Rake()
for index, row in df.iterrows():
  r.extract_keywords_from_text(row['description'])   # to extract key words, default in lower case
  key_words_dict_scores = r.get_word_degrees()   
  row['Key_words'] = list(key_words_dict_scores.keys())   # to assign list of key words to new column


df.head()


,isbn10,title,authors,thumbnail,description,categories,Key_words
0,0002005883,Gilead,Marilynne Robinson,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,Fiction,"[tale, best, friend, ’, life, style, sons, pit..."
1,0002261987,Spider's Web,Charles Osborne;Agatha Christie,http://books.google.com/books/content?id=gA5GP...,A new Christie for Christmas a fulllength nov...,Detective and mystery stories,"[ones, superb, storytelling, opened, first, dr..."
2,0006163831,The One Tree,Stephen R. Donaldson,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldsons acclaimed sec...,American fiction,"[volume, two, stephen, donaldsons, acclaimed, ..."
3,0006178731,Rage of angels,Sidney Sheldon,http://books.google.com/books/content?id=FKo2T...,A memorable mesmerizing heroine Jennifer bril...,Fiction,"[hate, mafias, schemes, win, organized, crime,..."
4,0006280897,The Four Loves,Clive Staples Lewis,http://books.google.com/books/content?id=XhQ5X...,Lewis work on the nature of love divides love ...,Christian life,"[natural, loves, become, distorted, without, l..."


In [125]:
#adding the categories i.e. genres to keywords as well so that similar genre books can be selected
df['categories']= df['categories'].apply(lambda x:x.split())
df['categories'].astype(str)
df['Key_words']=df['categories']+df['Key_words']
df.head()


,isbn10,title,authors,thumbnail,description,categories,Key_words
0,0002005883,Gilead,Marilynne Robinson,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,[Fiction],"[Fiction, tale, best, friend, ’, life, style, ..."
1,0002261987,Spider's Web,Charles Osborne;Agatha Christie,http://books.google.com/books/content?id=gA5GP...,A new Christie for Christmas a fulllength nov...,"[Detective, and, mystery, stories]","[Detective, and, mystery, stories, ones, super..."
2,0006163831,The One Tree,Stephen R. Donaldson,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldsons acclaimed sec...,"[American, fiction]","[American, fiction, volume, two, stephen, dona..."
3,0006178731,Rage of angels,Sidney Sheldon,http://books.google.com/books/content?id=FKo2T...,A memorable mesmerizing heroine Jennifer bril...,[Fiction],"[Fiction, hate, mafias, schemes, win, organize..."
4,0006280897,The Four Loves,Clive Staples Lewis,http://books.google.com/books/content?id=XhQ5X...,Lewis work on the nature of love divides love ...,"[Christian, life]","[Christian, life, natural, loves, become, dist..."


**making authors names into keywords as well, each author's name will be merged(firstnamelastname) and added as a separate keyword. **

In [126]:
df['authors']= df['authors'].apply(lambda x:x.lower().replace(' ',''))

In [127]:
df.head()

,isbn10,title,authors,thumbnail,description,categories,Key_words
0,0002005883,Gilead,marilynnerobinson,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,[Fiction],"[Fiction, tale, best, friend, ’, life, style, ..."
1,0002261987,Spider's Web,charlesosborne;agathachristie,http://books.google.com/books/content?id=gA5GP...,A new Christie for Christmas a fulllength nov...,"[Detective, and, mystery, stories]","[Detective, and, mystery, stories, ones, super..."
2,0006163831,The One Tree,stephenr.donaldson,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldsons acclaimed sec...,"[American, fiction]","[American, fiction, volume, two, stephen, dona..."
3,0006178731,Rage of angels,sidneysheldon,http://books.google.com/books/content?id=FKo2T...,A memorable mesmerizing heroine Jennifer bril...,[Fiction],"[Fiction, hate, mafias, schemes, win, organize..."
4,0006280897,The Four Loves,clivestapleslewis,http://books.google.com/books/content?id=XhQ5X...,Lewis work on the nature of love divides love ...,"[Christian, life]","[Christian, life, natural, loves, become, dist..."


In [128]:
df['authors']= df['authors'].apply(lambda x:x.lower().replace(';',' '))
df.head()

,isbn10,title,authors,thumbnail,description,categories,Key_words
0,0002005883,Gilead,marilynnerobinson,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,[Fiction],"[Fiction, tale, best, friend, ’, life, style, ..."
1,0002261987,Spider's Web,charlesosborne agathachristie,http://books.google.com/books/content?id=gA5GP...,A new Christie for Christmas a fulllength nov...,"[Detective, and, mystery, stories]","[Detective, and, mystery, stories, ones, super..."
2,0006163831,The One Tree,stephenr.donaldson,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldsons acclaimed sec...,"[American, fiction]","[American, fiction, volume, two, stephen, dona..."
3,0006178731,Rage of angels,sidneysheldon,http://books.google.com/books/content?id=FKo2T...,A memorable mesmerizing heroine Jennifer bril...,[Fiction],"[Fiction, hate, mafias, schemes, win, organize..."
4,0006280897,The Four Loves,clivestapleslewis,http://books.google.com/books/content?id=XhQ5X...,Lewis work on the nature of love divides love ...,"[Christian, life]","[Christian, life, natural, loves, become, dist..."


In [129]:
df

,isbn10,title,authors,thumbnail,description,categories,Key_words
0,0002005883,Gilead,marilynnerobinson,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,[Fiction],"[Fiction, tale, best, friend, ’, life, style, ..."
1,0002261987,Spider's Web,charlesosborne agathachristie,http://books.google.com/books/content?id=gA5GP...,A new Christie for Christmas a fulllength nov...,"[Detective, and, mystery, stories]","[Detective, and, mystery, stories, ones, super..."
2,0006163831,The One Tree,stephenr.donaldson,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldsons acclaimed sec...,"[American, fiction]","[American, fiction, volume, two, stephen, dona..."
3,0006178731,Rage of angels,sidneysheldon,http://books.google.com/books/content?id=FKo2T...,A memorable mesmerizing heroine Jennifer bril...,[Fiction],"[Fiction, hate, mafias, schemes, win, organize..."
4,0006280897,The Four Loves,clivestapleslewis,http://books.google.com/books/content?id=XhQ5X...,Lewis work on the nature of love divides love ...,"[Christian, life]","[Christian, life, natural, loves, become, dist..."
...,...,...,...,...,...,...,...
6803,8173031010,Journey to the East,hermannhesse,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,"[Adventure, stories]","[Adventure, stories, tale, 1932, enlightenment..."
6804,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,robinsharma,http://books.google.com/books/content?id=c_7mf...,Wisdom to Create a Life of Passion Purpose and...,"[Health, &, Fitness]","[Health, &, Fitness, wisdom, confront, peace, ..."
6805,8185300534,I Am that,srinisargadattamaharaj sudhakars.dikshit,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,[Philosophy],"[Philosophy, helps, suffering, unique, teacher..."
6808,9027712050,The Berlin Phenomenology,georgwilhelmfriedrichhegel,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegels Philos...,[History],"[History, logical, thoroughly, reliable, ba, s..."


In [130]:
df['authors']= df['authors'].apply(lambda x:x.split())
df['authors'].astype(str)
df['Key_words']=df['authors']+df['Key_words']
df.head()


,isbn10,title,authors,thumbnail,description,categories,Key_words
0,0002005883,Gilead,[marilynnerobinson],http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,[Fiction],"[marilynnerobinson, Fiction, tale, best, frien..."
1,0002261987,Spider's Web,"[charlesosborne, agathachristie]",http://books.google.com/books/content?id=gA5GP...,A new Christie for Christmas a fulllength nov...,"[Detective, and, mystery, stories]","[charlesosborne, agathachristie, Detective, an..."
2,0006163831,The One Tree,[stephenr.donaldson],http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldsons acclaimed sec...,"[American, fiction]","[stephenr.donaldson, American, fiction, volume..."
3,0006178731,Rage of angels,[sidneysheldon],http://books.google.com/books/content?id=FKo2T...,A memorable mesmerizing heroine Jennifer bril...,[Fiction],"[sidneysheldon, Fiction, hate, mafias, schemes..."
4,0006280897,The Four Loves,[clivestapleslewis],http://books.google.com/books/content?id=XhQ5X...,Lewis work on the nature of love divides love ...,"[Christian, life]","[clivestapleslewis, Christian, life, natural, ..."


In [131]:
#creating the bag of words which is used by the rake function to get vectors
df['Bag_of_words'] = ''
for index, row in df.iterrows():
    words = ''
    words += ' '.join(row['Key_words']) + ' '
    row['Bag_of_words'] = words
df['Bag_of_words'] = df['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')


In [132]:
df.head(7)

,isbn10,title,authors,thumbnail,description,categories,Key_words,Bag_of_words
0,0002005883,Gilead,[marilynnerobinson],http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,[Fiction],"[marilynnerobinson, Fiction, tale, best, frien...",marilynnerobinson Fiction tale best friend ’ l...
1,0002261987,Spider's Web,"[charlesosborne, agathachristie]",http://books.google.com/books/content?id=gA5GP...,A new Christie for Christmas a fulllength nov...,"[Detective, and, mystery, stories]","[charlesosborne, agathachristie, Detective, an...",charlesosborne agathachristie Detective and my...
2,0006163831,The One Tree,[stephenr.donaldson],http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldsons acclaimed sec...,"[American, fiction]","[stephenr.donaldson, American, fiction, volume...",stephenr.donaldson American fiction volume two...
3,0006178731,Rage of angels,[sidneysheldon],http://books.google.com/books/content?id=FKo2T...,A memorable mesmerizing heroine Jennifer bril...,[Fiction],"[sidneysheldon, Fiction, hate, mafias, schemes...",sidneysheldon Fiction hate mafias schemes win ...
4,0006280897,The Four Loves,[clivestapleslewis],http://books.google.com/books/content?id=XhQ5X...,Lewis work on the nature of love divides love ...,"[Christian, life]","[clivestapleslewis, Christian, life, natural, ...",clivestapleslewis Christian life natural loves...
5,0006280935,The Problem of Pain,[clivestapleslewis],http://books.google.com/books/content?id=Kk-uV...,In The Problem of Pain CS Lewis one of the mos...,"[Christian, life]","[clivestapleslewis, Christian, life, renowned,...",clivestapleslewis Christian life renowned chri...
6,0006353282,An Autobiography,[agathachristie],http://books.google.com/books/content?id=c49GQ...,Donation,"[Authors,, English]","[agathachristie, Authors,, English, donation]","agathachristie Authors, English donation"


In [133]:
#trial
df['Bag_of_words'][3]


'sidneysheldon Fiction hate mafias schemes win organized crime dangerous dramatic world implacable enemy love flashbulb lit courtrooms ambitious prosecutors begin way hatred destructive memorable mesmerizing heroine jennifer brilliant beautiful climb dark arena political power attorney'

In [134]:
df=df.reset_index()

In [135]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])
count_matrix


<6247x38388 sparse matrix of type '<class 'numpy.int64'>'
	with 237350 stored elements in Compressed Sparse Row format>

In [136]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.01059998 0.02649995 ... 0.03623939 0.00979968 0.        ]
 [0.01059998 1.         0.025      ... 0.03418817 0.01849001 0.02581989]
 [0.02649995 0.025      1.         ... 0.         0.02311251 0.        ]
 ...
 [0.03623939 0.03418817 0.         ... 1.         0.03160698 0.02942449]
 [0.00979968 0.01849001 0.02311251 ... 0.03160698 1.         0.        ]
 [0.         0.02581989 0.         ... 0.02942449 0.         1.        ]]


In [137]:
indices = pd.Series(df['isbn10'])
indices[:5]


0    0002005883
1    0002261987
2    0006163831
3    0006178731
4    0006280897
Name: isbn10, dtype: object

In [138]:
df

,index,isbn10,title,authors,thumbnail,description,categories,Key_words,Bag_of_words
0,0,0002005883,Gilead,[marilynnerobinson],http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,[Fiction],"[marilynnerobinson, Fiction, tale, best, frien...",marilynnerobinson Fiction tale best friend ’ l...
1,1,0002261987,Spider's Web,"[charlesosborne, agathachristie]",http://books.google.com/books/content?id=gA5GP...,A new Christie for Christmas a fulllength nov...,"[Detective, and, mystery, stories]","[charlesosborne, agathachristie, Detective, an...",charlesosborne agathachristie Detective and my...
2,2,0006163831,The One Tree,[stephenr.donaldson],http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldsons acclaimed sec...,"[American, fiction]","[stephenr.donaldson, American, fiction, volume...",stephenr.donaldson American fiction volume two...
3,3,0006178731,Rage of angels,[sidneysheldon],http://books.google.com/books/content?id=FKo2T...,A memorable mesmerizing heroine Jennifer bril...,[Fiction],"[sidneysheldon, Fiction, hate, mafias, schemes...",sidneysheldon Fiction hate mafias schemes win ...
4,4,0006280897,The Four Loves,[clivestapleslewis],http://books.google.com/books/content?id=XhQ5X...,Lewis work on the nature of love divides love ...,"[Christian, life]","[clivestapleslewis, Christian, life, natural, ...",clivestapleslewis Christian life natural loves...
...,...,...,...,...,...,...,...,...,...
6242,6803,8173031010,Journey to the East,[hermannhesse],http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,"[Adventure, stories]","[hermannhesse, Adventure, stories, tale, 1932,...",hermannhesse Adventure stories tale 1932 enlig...
6243,6804,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,[robinsharma],http://books.google.com/books/content?id=c_7mf...,Wisdom to Create a Life of Passion Purpose and...,"[Health, &, Fitness]","[robinsharma, Health, &, Fitness, wisdom, conf...",robinsharma Health & Fitness wisdom confront p...
6244,6805,8185300534,I Am that,"[srinisargadattamaharaj, sudhakars.dikshit]",http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,[Philosophy],"[srinisargadattamaharaj, sudhakars.dikshit, Ph...",srinisargadattamaharaj sudhakars.dikshit Philo...
6245,6808,9027712050,The Berlin Phenomenology,[georgwilhelmfriedrichhegel],http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegels Philos...,[History],"[georgwilhelmfriedrichhegel, History, logical,...",georgwilhelmfriedrichhegel History logical tho...


**we are searching the book by isbn10 here but in the final code will use fuzzywuzzy to search through entering book title like done in the colab_filter **

In [139]:
def recommend(isbn, cosine_sim = cosine_sim):
    recommended_books = []
    resdf=pd.DataFrame()
    idx = indices[indices == isbn].index[0]   # to get the index of the movie title matching the input movie
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)  
    top_10_indices = list(score_series.iloc[1:11].index)   # to get the indices of top 10 most similar movies
    # [1:11] to exclude 0 (index 0 is the input movie itself)
    for i in top_10_indices:   # to append the titles of top 10 similar movies to the recommended_movies list
        recommended_books.append(list(df['isbn10'])[i])
    resdf=df[df['isbn10'].isin(recommended_books)]  
    return resdf

recommend('0439064864')# try 0439064864 for harry potter 2, 0439064864 is the time machine by hg wells

#df.loc[df['isbn10'] == '0451528557']
#print(df['title'][2510])#so duplicate book titles exist!

,index,isbn10,title,authors,thumbnail,description,categories,Key_words,Bag_of_words
454,473,0064400964,Betsy-Tacy,[maudhartlovelace],http://books.google.com/books/content?id=tfUCG...,Best Friends Forever There are lots of childre...,"[Juvenile, Fiction]","[maudhartlovelace, Juvenile, Fiction, wonderfu...",maudhartlovelace Juvenile Fiction wonderful su...
2503,2676,0439358078,Harry Potter and the Order of the Phoenix (Boo...,"[rowling,j.k.]",http://books.google.com/books/content?id=OIJ5B...,In Harry Potter and the Order of the Phoenix L...,"[Juvenile, Fiction]","[rowling,j.k., Juvenile, Fiction, ministry, lo...","rowling,j.k. Juvenile Fiction ministry looming..."
2519,2697,0439554896,Harry Potter and the Chamber of Secrets,"[j.k.rowling, marygrandpre]",http://books.google.com/books/content?id=h2Y-P...,When the Chamber of Secrets is opened again at...,"[Juvenile, Fiction]","[j.k.rowling, marygrandpre, Juvenile, Fiction,...",j.k.rowling marygrandpre Juvenile Fiction dark...
2520,2698,0439554934,Harry Potter and the Sorcerer's Stone (Book 1),"[rowling,j.k.]",http://books.google.com/books/content?id=9jA0B...,In Harry Potter and the Sorcerers Stone Harry ...,"[Juvenile, Fiction]","[rowling,j.k., Juvenile, Fiction, closest, fri...","rowling,j.k. Juvenile Fiction closest friends ..."
2531,2710,043965548X,Harry Potter and the Prisoner of Azkaban (Book 3),"[rowling,j.k.]",http://books.google.com/books/content?id=IZN5B...,For twelve long years the dread fortress of Az...,"[Juvenile, Fiction]","[rowling,j.k., Juvenile, Fiction, father, ’, y...","rowling,j.k. Juvenile Fiction father ’ youknow..."
2533,2712,0439682584,Harry Potter,[j.k.rowling],http://books.google.com/books/content?id=DAAAA...,The first five years of Harry Potter magic are...,"[Juvenile, Fiction]","[j.k.rowling, Juvenile, Fiction, azkaban, phoe...",j.k.rowling Juvenile Fiction azkaban phoenix s...
2542,2723,0439785960,Harry Potter and the Half-Blood Prince (Book 6),"[rowling,j.k.]",http://books.google.com/books/content?id=QzI0B...,When Harry Potter and the HalfBlood Prince ope...,"[Juvenile, Fiction]","[rowling,j.k., Juvenile, Fiction, named, tom, ...","rowling,j.k. Juvenile Fiction named tom riddle..."
2548,2730,0439827604,The Harry Potter Collection,[j.k.rowling],http://books.google.com/books/content?id=v0z1P...,The first six years of Harry Potter magic are ...,"[Juvenile, Fiction]","[j.k.rowling, Juvenile, Fiction, azkaban, orde...",j.k.rowling Juvenile Fiction azkaban order hal...
4169,4508,074754624X,Harry Potter and the Goblet of Fire,[j.k.rowling],http://books.google.com/books/content?id=5eSxn...,The summer holidays are dragging on and Harry ...,"[Juvenile, Fiction]","[j.k.rowling, Juvenile, Fiction, yet, preparin...",j.k.rowling Juvenile Fiction yet preparing har...
5433,5911,1416909052,Amelia's Notebook,[marissamoss],http://books.google.com/books/content?id=szoDb...,The handlettered contents of a nineyearold gir...,"[Juvenile, Fiction]","[marissamoss, Juvenile, Fiction, older, sister...",marissamoss Juvenile Fiction older sister also...


**conclusion**
we can see all of these are similar movies, share categories and from the description we can tell that they are science fiction books and have the author H.G. wells in common in one some too. 

**trying to find how many books are in common in Books and books7k**


In [140]:
bdf = pd.read_csv('/content/drive/MyDrive/Books.csv', engine='python', error_bad_lines=False)
bdf.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [141]:
in_common=df[df['isbn10'].isin(list(bdf['ISBN'].unique()))]
in_common.shape

(2265, 9)

this means that we can only use books out of 2265 books in collaborative filtering or to search. 